In the Notebook we will read 4 tables that come from the AdventureWorks database; FactInternetSales, DimProduct, DimSalesTerritory, and DimCurrency.

In [0]:
#Load Data
options = {
    "header": "true",
    "inferSchema": "true",
    "delimiter": ";"
}
dim_product =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables/DimProduct-1.csv")
fact_int_sales =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables/FactInternetSales-2.csv")
dim_sales_terr =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables/DimSalesTerritory.csv")
dim_curreny = spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables/DimCurrency.csv")  


# Fact Internet Sales Table

In [0]:
display(fact_int_sales.limit(10))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate
310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,null,null,413.15,413.15,699.1,55.93,174.775,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06
310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06
351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,3374.99,3374.99,1898.09,1898.09,3374.99,270.0,843.748,2010-12-30,2011-01-11,2011-01-06
344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-30,2011-01-11,2011-01-06
312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07


In [0]:
from pyspark.sql.functions import count
display(fact_int_sales.count())

60399

In [0]:
from pyspark.sql.functions import col
display(fact_int_sales.filter(col('SalesAmount').isNull()))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,RevisionNumber,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate


In [0]:
# We Exclude the rows where the Sales amount is null
fact_int_sales = fact_int_sales.filter(~col('SalesAmount').isNull())

In [0]:
display(fact_int_sales.describe().toPandas())

summary,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,RevisionNumber,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber
count,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835,58835
mean,440.26474037562673,2.0130174444072407E7,2.013054109830883E7,2.0130378132523157E7,18862.503169881875,1.0429506246281974,69.95502676977989,6.240809042236764,null,1.9098665760176765,1.0,1.0,372.7583104237543,372.7583104237543,0.0,0.0,159.96771529700865,159.96771529700865,373.3995207187005,70.63716587507815,104.12320907622595,null,null
stddev,118.35815588551966,3898.8546341686433,4197.33564309059,4026.6373309270925,5428.33439688519,0.34705683150388067,42.08295012851483,2.961039857308672,null,1.019284283354953,0.0,0.0,801.2433316548457,801.2433316548457,0.0,0.0,337.1614479809296,337.1614479809296,800.5862158809779,169.69676510307156,212.2750590770042,null,null
min,214,20101229,20110110,20110105,11000,1,6,1,SO43697,1,1,1,1120.49,1120.49,0,0.0,0.8565,0.8565,1120.49,0.1832,0.0573,NULL,NULL
max,606,20140128,20140209,20140204,29483,14,100,10,SO75123,8,1,1,9.99,9.99,0,0.0,93.463,93.463,9.99,97.188,894.568,NULL,NULL


In [0]:
# DiscountAmount & UnitPriceDiscountPct columns have zeros only so we drop them
# OrderQuantity & RevisionNumber only have the value 1 so we drop them since it will not be beneficial in analysis 
# CustomerPONumber & CarrierTrackingNumber have Nulls only so we drop them 
columns_drop = ["DiscountAmount", "UnitPriceDiscountPct", "OrderQuantity", "RevisionNumber", "CustomerPONumber", "CarrierTrackingNumber"]
fact_int_sales = fact_int_sales.drop(*columns_drop)


In [0]:
# Convert the numeric column to float instead of string 
# ShipDate, DueDate and OrderDate Columns are of type timestamp, we have to convert them to date
from pyspark.sql.functions import  regexp_replace
fact_int_sales = (fact_int_sales
       .withColumn("ProductKey", col("ProductKey").cast("int"))
      .withColumn("ExtendedAmount",col("ExtendedAmount").cast("float"))
      .withColumn("UnitPrice", col("UnitPrice").cast("float"))
      .withColumn("TotalProductCost",col("TotalProductCost").cast("float"))
      .withColumn("TaxAmt", col("TaxAmt").cast("float"))
      .withColumn("Freight", col("Freight").cast("float"))
      .withColumn("SalesAmount", col("SalesAmount").cast("float"))
      .withColumn("ProductStandardCost",col("ProductStandardCost").cast("float"))
      .withColumn("ShipDate", col("ShipDate").cast("date"))
      .withColumn("DueDate", col("DueDate").cast("date"))
      .withColumn("OrderDate", col("OrderDate").cast("date"))
      )

In [0]:
# Checking the scehma  
fact_int_sales.printSchema() 

root
 |-- ProductKey: integer (nullable = true)
 |-- OrderDateKey: string (nullable = true)
 |-- DueDateKey: integer (nullable = true)
 |-- ShipDateKey: integer (nullable = true)
 |-- CustomerKey: integer (nullable = true)
 |-- PromotionKey: integer (nullable = true)
 |-- CurrencyKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- SalesOrderLineNumber: integer (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- ExtendedAmount: float (nullable = true)
 |-- ProductStandardCost: float (nullable = true)
 |-- TotalProductCost: float (nullable = true)
 |-- SalesAmount: float (nullable = true)
 |-- TaxAmt: float (nullable = true)
 |-- Freight: float (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- DueDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)



# Dim Product Table

In [0]:
display(dim_product.limit(10))

ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,ListPrice,Size,SizeRange,Weight,DaysToManufacture,ProductLine,DealerPrice,Class,Style,ModelName,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
1,AR-5381,NULL,NULL,NULL,Adjustable Race,null,null,NULL,0,NA,1000,750,NULL,NULL,NA,NULL,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
2,BA-8327,NULL,NULL,NULL,Bearing Ball,null,null,NULL,0,NA,1000,750,NULL,NULL,NA,NULL,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
3,BE-2349,NULL,NULL,NULL,BB Ball Bearing,null,null,NULL,0,NA,800,600,NULL,NULL,NA,NULL,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
4,BE-2908,NULL,NULL,NULL,Headset Ball Bearings,null,null,NULL,0,NA,800,600,NULL,NULL,NA,NULL,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
5,BL-2036,NULL,NULL,NULL,Blade,null,null,NULL,0,NA,800,600,NULL,NULL,NA,NULL,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
6,CA-5965,NULL,NULL,NULL,LL Crankarm,null,null,NULL,0,Black,500,375,NULL,NULL,NA,NULL,0,NULL,NULL,L,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
7,CA-6738,NULL,NULL,NULL,ML Crankarm,null,null,NULL,0,Black,500,375,NULL,NULL,NA,NULL,0,NULL,NULL,M,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
8,CA-7457,NULL,NULL,NULL,HL Crankarm,null,null,NULL,0,Black,500,375,NULL,NULL,NA,NULL,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
9,CB-2903,NULL,NULL,NULL,Chainring Bolts,null,null,NULL,0,Silver,1000,750,NULL,NULL,NA,NULL,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current
10,CN-6137,NULL,NULL,NULL,Chainring Nut,null,null,NULL,0,Silver,1000,750,NULL,NULL,NA,NULL,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2003-07-01T00:00:00.000+0000,NULL,Current


In [0]:
display(dim_product.describe().toPandas())

summary,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,ListPrice,Size,SizeRange,Weight,DaysToManufacture,ProductLine,DealerPrice,Class,Style,ModelName,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,EndDate,Status
count,606,606,606,606,606,606,319,319,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606,606
mean,303.5,null,12.128463476070529,null,null,null,null,null,770.6296500000001,0.6551155115511551,null,495.2013201320132,371.4009900990099,713.5176867088601,49.05511811023622,70.0,56.193226950354635,1.2013201320132014,null,1430318.023088605,null,null,null,null,null,null,null,null,null,null,null,null,null,null
stddev,175.08140963563207,null,9.423328075954464,null,null,null,null,null,518.4640339182464,0.47572327172751,null,364.00711426828434,273.0053357012136,866.4272704789952,7.389028386725068,null,158.00065246682107,1.5088931655188196,null,2783637.4650102835,null,null,null,null,null,null,null,null,null,null,null,null,null,null
min,1,AR-5381,1,G,CM,AWC Logo Cap,Bielas GA,Antivol câble,0.8565,0,Black,4,3,1.093.364,38,38-40 CM,1000,0,M,1020594,H,M,All-Purpose Bike Stand,"""Carries 4 bikes securely; steel construction, fits 2"""" receiver hitch.""",Acier chromé.,HL 铝制车架经过特别打造，不仅外形悦目而且坚固耐用；不论何时何地，日常骑乘，轻松掌控。男用自行车。,NULL,"""אופני מבוגרים למתחילים; מציעים רכיבה נוחה """"מחוף לחוף"""" או לאורך הרחוב. טבורים וחישורים לשחרור מהיר.""",NULL,Aerodynamische Felgen für ein geschmeidiges Fahrerlebnis.,1 日中乗っても快適なアナトミック デザインで、耐久性のある皮製。,"""Yüksek kaliteli 1"""" furş takımı, hızlı yağlama için yağ kapaklı.""",2007-12-28 00:00:00.000,Current
max,606,WB-H098,NULL,NULL,NULL,"Women's Tights, S",Soporte multiusos para bicicletas,"Vélo de route 750 noir, 58",NULL,1,Yellow,1000,750,NULL,weight,XL,NULL,4,T,NULL,NULL,W,Women's Tights,"Wraps to fit front and rear tires, carrier and 2 keys included.","Équipé de la même technologie que tous nos vélos de route, avec un cadre femmes. Idéal pour la promenade ou la course sur route.",高质量的一英寸无螺纹车头碗组具有油口，可确保快速润滑。,يوفر هيكل الألومنيوم سميك الأطراف خفيف الوزن وضع قيادة أكثر انتصابًا أثناء القيام برحلات في أرجاء البلدة. يوفر تصميمنا الابتكاري أعلى درجات الراحة.,תאורה במחיר סביר לרכיבה לילית בטוחה – פועלת באמצעות סוללת AAA,ไฟหน้าแบบทนทาน กันน้ำ,Wiederaufladbares Vorderlicht mit zwei Strahlen.,高速走行用のステンレス製スポークを備えたアルミニウム合金リム。,Şoku emen ve daha hassas yönlendirilebilen güçlü tasarım.,NULL,NULL


In [0]:
# Drop the columns that we will not need in our analysis
columns_drop = ["EnglishProductName", "SpanishProductName", "FrenchProductName","FinishedGoodsFlag",
"EnglishDescription","FrenchDescription","ChineseDescription","ArabicDescription","HebrewDescription","ThaiDescription","GermanDescription","JapaneseDescription","TurkishDescription","ProductSubcategoryKey"]
dim_product = dim_product.drop(*columns_drop)


In [0]:
dim_product = (dim_product
      .withColumn("StandardCost", col("StandardCost").cast("float"))    
      .withColumn("DealerPrice", col("DealerPrice").cast("float"))
      .withColumn("weight",col("weight").cast("float"))
      .withColumn("StartDate", col("StartDate").cast("date"))
      )


In [0]:
from pyspark.sql.functions import col, count, when,isnan
# Count nulls for each column
display(dim_product.select(
    [count(
        when((col(c).isNull()) | (col(c) =='NULL'), c)
        ).alias(c) for c in dim_product.columns]
    ))

ProductKey,ProductAlternateKey,WeightUnitMeasureCode,SizeUnitMeasureCode,StandardCost,Color,SafetyStockLevel,ReorderPoint,ListPrice,Size,SizeRange,weight,DaysToManufacture,ProductLine,DealerPrice,Class,Style,ModelName,StartDate,EndDate,Status
0,0,324,353,596,0,0,0,211,306,0,324,0,226,211,276,305,209,0,406,200


In [0]:
# Drop the columns with the null values majority
null_columns_drop = ["StandardCost","weight","DearlerPrice","Style","ProductLine","Size","SizeUnitMeasureCode","WeightUnitMeasureCode","Class","Status","ListPrice","DealerPrice","ModelName"]
dim_product = dim_product.drop(*null_columns_drop)
dim_product.columns

Out[20]: ['ProductKey',
 'ProductAlternateKey',
 'Color',
 'SafetyStockLevel',
 'ReorderPoint',
 'SizeRange',
 'DaysToManufacture',
 'StartDate',
 'EndDate']

In [0]:
dim_product.printSchema()

root
 |-- ProductKey: integer (nullable = true)
 |-- ProductAlternateKey: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: integer (nullable = true)
 |-- ReorderPoint: integer (nullable = true)
 |-- SizeRange: string (nullable = true)
 |-- DaysToManufacture: integer (nullable = true)
 |-- StartDate: date (nullable = true)
 |-- EndDate: string (nullable = true)



# DimSalesTerritory Table

In [0]:
display(dim_sales_terr)

SalesTerritoryKey,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup
1,1,Northwest,United States,North America
2,2,Northeast,United States,North America
3,3,Central,United States,North America
4,4,Southwest,United States,North America
5,5,Southeast,United States,North America
6,6,Canada,Canada,North America
7,7,France,France,Europe
8,8,Germany,Germany,Europe
9,9,Australia,Australia,Pacific
10,10,United Kingdom,United Kingdom,Europe


In [0]:
# Drop the 11th key since it has no values 
dim_sales_terr = dim_sales_terr[dim_sales_terr['SalesTerritoryKey'] != 11]

In [0]:
dim_sales_terr.printSchema()

root
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- SalesTerritoryAlternateKey: integer (nullable = true)
 |-- SalesTerritoryRegion: string (nullable = true)
 |-- SalesTerritoryCountry: string (nullable = true)
 |-- SalesTerritoryGroup: string (nullable = true)



# DimCurrency Table

In [0]:
display(dim_curreny)

CurrencyKey,CurrencyAlternateKey,CurrencyName
1,AFA,Afghani
2,DZD,Algerian Dinar
3,ARS,Argentine Peso
4,AMD,Armenian Dram
5,AWG,Aruban Guilder
6,AUD,Australian Dollar
7,AZM,Azerbaijanian Manat
8,BSD,Bahamian Dollar
9,BHD,Bahraini Dinar
10,THB,Baht
